In [6]:
import requests
from bs4 import BeautifulSoup

# 서울 지역 116곳을 저장할 리스트
area_list = []

# API 키
api_key = '6e51654f776b353335387168745664'

# 서울 지역 데이터 크롤링
for i in range(1, 117):
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/1000/POI{i:03d}'
    response = requests.get(url)
    xml_data = response.text
    soup = BeautifulSoup(xml_data, 'xml')
    
    # 지역명 가져오기
    area_name_tag = soup.find('AREA_NM')
    
    if area_name_tag:
        area_name = area_name_tag.text.strip()
        area_list.append(area_name)

# 제외할 지역 리스트
excluded_areas = {"발산역", "수유역", "양재역", "연신내역", "북한산우이역", "신논현역·논현역", "회기역", "동대문 관광특구"}

# 불필요한 지역 제거 (리스트에서 제외)
area_list = [area for area in area_list if area not in excluded_areas]

# 최종 결과 출력
print("저장된 지역 목록 (제외된 지역 삭제 후):", area_list)

저장된 지역 목록 (제외된 지역 삭제 후): ['강남 MICE 관광특구', '명동 관광특구', '이태원 관광특구', '잠실 관광특구', '종로·청계 관광특구', '홍대 관광특구', '경복궁', '광화문·덕수궁', '보신각', '서울 암사동 유적', '창덕궁·종묘', '가산디지털단지역', '강남역', '건대입구역', '고덕역', '고속터미널역', '교대역', '구로디지털단지역', '구로역', '군자역', '남구로역', '대림역', '동대문역', '뚝섬역', '미아사거리역', '사당역', '삼각지역', '서울대입구역', '서울식물원·마곡나루역', '서울역', '선릉역', '성신여대입구역', '신도림역', '신림역', '신촌·이대역', '역삼역', '오목교역·목동운동장', '왕십리역', '용산역', '이태원역', '장지역', '장한평역', '천호역', '총신대입구(이수)역', '충정로역', '합정역', '혜화역', '홍대입구역(2호선)', '4·19 카페거리', '가락시장', '가로수길', '광장(전통)시장', '김포공항', '낙산공원·이화마을', '노량진', '덕수궁길·정동길', '방배역 먹자골목', '북촌한옥마을', '서촌', '성수카페거리', '수유리 먹자골목', '쌍문동 맛집거리', '압구정로데오거리', '여의도', '연남동', '영등포 타임스퀘어', '외대앞', '용리단길', '이태원 앤틱가구거리', '인사동', '창동 신경제 중심지', '청담동 명품거리', '청량리 제기동 일대 전통시장', '해방촌·경리단길', 'DDP(동대문디자인플라자)', 'DMC(디지털미디어시티)', '강서한강공원', '고척돔', '광나루한강공원', '광화문광장', '국립중앙박물관·용산가족공원', '난지한강공원', '남산공원', '노들섬', '뚝섬한강공원', '망원한강공원', '반포한강공원', '북서울꿈의숲', '불광천', '서리풀공원·몽마르뜨공원', '서울광장', '서울대공원', '서울숲공원', '아차산', '양화한강공원', '어린이대공원', '여의도한강공원', '월드컵공원', '

In [13]:
import mysql.connector

ojk = mysql.connector.connect(
    host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "eda",
    password = "ojk0707",
    database = "ojk",
    autocommit=True
)
mycursor = ojk.cursor()

In [14]:
mycursor = ojk.cursor()
mycursor.execute("desc enjoy")
result = mycursor.fetchall()
for data in result:
    print(data)

('place', 'varchar(100)', 'NO', 'PRI', None, '')
('region', 'varchar(16)', 'YES', '', None, '')
('review', 'int', 'YES', '', None, '')
('address', 'varchar(255)', 'YES', '', None, '')
('latitude', 'decimal(10,6)', 'YES', '', None, '')
('longitude', 'decimal(10,6)', 'YES', '', None, '')


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import urllib.parse
import time
import re
import requests

KAKAO_API_KEY = "1a6063efba1d306ac8725b784b11fa13"

def switch_left():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH, '//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)

def switch_right():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH, '//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

def clean_address(address):
    address = re.sub(r'\s*\(우\)\d{5}.*$', '', address)
    match = re.search(r'([가-힣\d\s-]*로\d*(길\s?\d+)?\s?\d*-?\d*)', address)
    address = match.group(0) if match else address.strip()
    address = re.sub(r'(\s*\d*\s*층.*$)|(\s+\d+\s*$)', '', address)
    return address.strip()

def extract_number(text):
    match = re.search(r'\d+', text.replace(',', ''))
    return int(match.group(0)) if match else 0

def get_lat_lon(address):
    time.sleep(1)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={address}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 401:
        return None, None
    elif response.status_code == 429:
        time.sleep(60)
        return get_lat_lon(address)

    result = response.json()
    if "documents" not in result or not result["documents"]:
        return None, None

    match_first = result["documents"][0]["address"]
    lat = float(match_first["y"])
    lon = float(match_first["x"])
    return lat, lon

# Selenium 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(3)

for place in area_list:
    search_url = f"https://map.naver.com/p/search/{urllib.parse.quote(place + ' 근처 놀거리')}"
    driver.get(search_url)
    time.sleep(3)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchIframe"]')))
        switch_left()
    except:
        print(f"[{place}] 검색결과 없음 - 스킵")
        continue

    try:
        scrollable_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container"))
        )
    except Exception as e:
        print(f"[{place}] 스크롤 요소 탐색 실패: {e} - 스킵")
        continue

    # 스크롤 내리기
    scroll_pause_time = 1
    max_scroll_attempts = 30
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    for _ in range(max_scroll_attempts):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

    elements = driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]//li')

    for index, element in enumerate(elements):
        try:
            switch_left()
            clickable = element.find_element(By.CLASS_NAME, 'YgcU0')
            clickable.click()
            time.sleep(2)
        except:
            print(f"{index+1}번째 장소 클릭 실패")
            continue

        try:
            switch_right()
            time.sleep(2)

            soup = BeautifulSoup(driver.page_source, "html.parser")

            title = soup.find("div", class_="LylZZ v8v5j")
            store_name = title.find("span", class_="GHAhO").get_text(strip=True) if title else "놀거리명 없음"
            category = title.find("span", class_="lnJFt").get_text(strip=True) if title else "카테고리 없음"

            # '카페,디저트' 카테고리 제외
            if category == "카페,디저트":
                continue

            visitor_review_element = soup.find("a", string=lambda text: text and "방문자 리뷰" in text)
            visitor_review = extract_number(visitor_review_element.get_text(strip=True)) if visitor_review_element else 0

            blog_review_element = soup.find("a", string=lambda text: text and "블로그 리뷰" in text)
            blog_review = extract_number(blog_review_element.get_text(strip=True)) if blog_review_element else 0

            total_reviews = visitor_review + blog_review

            address_element = soup.find("span", class_="LDgIH")
            address = clean_address(address_element.get_text(strip=True)) if address_element else "주소 없음"

            lat, lon = get_lat_lon(address)

            print(f"\n[지역: {place}] {index+1}. {store_name} · {category}")
            print(f"리뷰 수: {total_reviews}")
            print(f"주소: {address}")
            print(f"위도: {lat}, 경도: {lon}")
            print("-" * 50)

            mycursor.execute("""
                INSERT INTO restaurant
                VALUES(%s, %s, %s, %s, %s, %s)
            """, (store_name, place, total_reviews,  address, lat, lon))
            ojk.commit()

        except Exception as ex:
            print(f"{index+1}번째 장소 정보 수집 실패: {ex}")
            print("-" * 50)
            continue

driver.quit()


[강남 MICE 관광특구] 스크롤 요소 탐색 실패: Message: 
Stacktrace:
#0 0x64c1cc806ffa <unknown>
#1 0x64c1cc2c5970 <unknown>
#2 0x64c1cc317385 <unknown>
#3 0x64c1cc3175b1 <unknown>
#4 0x64c1cc3663c4 <unknown>
#5 0x64c1cc33d2bd <unknown>
#6 0x64c1cc36370c <unknown>
#7 0x64c1cc33d063 <unknown>
#8 0x64c1cc309328 <unknown>
#9 0x64c1cc30a491 <unknown>
#10 0x64c1cc7ce42b <unknown>
#11 0x64c1cc7d22ec <unknown>
#12 0x64c1cc7b5a22 <unknown>
#13 0x64c1cc7d2e64 <unknown>
#14 0x64c1cc799bef <unknown>
#15 0x64c1cc7f5558 <unknown>
#16 0x64c1cc7f5736 <unknown>
#17 0x64c1cc805e76 <unknown>
#18 0x77ddad894ac3 <unknown>
 - 스킵
[명동 관광특구] 스크롤 요소 탐색 실패: Message: 
Stacktrace:
#0 0x64c1cc806ffa <unknown>
#1 0x64c1cc2c5970 <unknown>
#2 0x64c1cc317385 <unknown>
#3 0x64c1cc3175b1 <unknown>
#4 0x64c1cc3663c4 <unknown>
#5 0x64c1cc33d2bd <unknown>
#6 0x64c1cc36370c <unknown>
#7 0x64c1cc33d063 <unknown>
#8 0x64c1cc309328 <unknown>
#9 0x64c1cc30a491 <unknown>
#10 0x64c1cc7ce42b <unknown>
#11 0x64c1cc7d22ec <unknown>
#12 0x64c1cc7b5a22

In [11]:
driver.quit()